## Computer Vision(CS 436, CS5310, EE513) Programming Assignment 3 Part 1

### Currently there are lots of professional cartoonizer applications available in the market but most of the them are not freeware, you don't need powerful rendering software or even years of experience to develop such an application(as you will see in this assignment) All you need is essentially a bilateral filter and some edge detection. You are allowed to use opencv for this assigment

## Step#1
### In this step we will be applying a bilateral filter on our input image. A bilateral filter is used for smoothening images and reducing noise, while preserving edges, because a bilateral filter smooths flat regions while keeping edges sharp, it is ideally suited to convert an RGB image into a cartoon. Unfortunately, bilateral filters are orders of magnitudes slower than other smoothing operators (e.g., Gaussian blur). Thus, if speed is important, it might be a good idea to operate on a down-scaled version of the original image first and then upscale it afterwards.

In [1]:
#required imports
import matplotlib
import numpy as np
from mpl_toolkits import mplot3d
from matplotlib import pyplot as plt
from matplotlib import cm
# from google.colab.patches import cv2_imshow
from matplotlib import image as mpimg
from matplotlib.pyplot import figure
import cv2
%matplotlib inline

Reading "input.jpg" through cv2 (BGR format):

In [2]:
Image = cv2.imread("input.jpg")

Converting image from BGR format to RGB:

In [3]:
RGBImage = cv2.cvtColor(Image, cv2.COLOR_BGR2RGB)

Using bilateral filter for edge-aware smoothing:


In [4]:
num_down = 2 # number of downsampling steps 

# downsample image using Gaussian pyramid(see opencv 'pyrDown()' function)
Smoothened = RGBImage
for x in range(num_down):
  Smoothened = cv2.pyrDown(Smoothened)


num_bilateral = 7 # number of bilateral filtering steps

FilterSize = 9
SigmaColor = 9
SigmaSpace = 7

# repeatedly apply small bilateral filter instead of applying one large filter
for x in range(num_bilateral):
  Smoothened = cv2.bilateralFilter(Smoothened, FilterSize, SigmaColor, SigmaSpace)

# upsample image to original size (see opencv 'pyrUp()' function)
for x in range(num_down):
  Smoothened = cv2.pyrUp(Smoothened)

## Step#2
### In this step we will blur the original image. This is considered as a pre-processing step before we move on towards the edge detection step. We will apply a median filter on the image, which replaces each pixel value with the median value of all the pixels in a small neighborhood.

Converting to grayscale:

In [5]:
GrayScaled = cv2.cvtColor(RGBImage, cv2.COLOR_RGB2GRAY)

Applying median blur:

In [6]:
KernelSize = 7
MedianBlurred = cv2.medianBlur(GrayScaled, KernelSize)

## Step#3

### In this step we will create an edge mask from the output produced in step#2 using adaptive thresholding 

Detecting and enhancing edges (see opencv 'adaptiveThreshold()' function):

In [7]:
MaximumValue = 255
AdaptiveMethod = cv2.ADAPTIVE_THRESH_MEAN_C
ThresholdType = cv2.THRESH_BINARY
BlockSize = 9
Constant = 2
Enhanced = cv2.adaptiveThreshold(MedianBlurred, MaximumValue, AdaptiveMethod, ThresholdType, BlockSize, Constant)

## Final Step

### In this step we will combine the output produced in step#1 and step#3 using a bitwise and operator to produce our final output.(Note: You need to convert output from step#3 to color first)

Converting back to color:

In [8]:
Colored = cv2.cvtColor(Enhanced, cv2.COLOR_GRAY2RGB)

Note: 'Smoothened' and 'Colored' might have distinct shapes.

Adjusting shape:

In [9]:
SmoothenedShape = Smoothened.shape
SmoothenedRows = SmoothenedShape[0]
SmoothenedColumns = SmoothenedShape[1]

ColoredShape = Colored.shape
ColoredRows = ColoredShape[0]
ColoredColumns = ColoredShape[1]

In [10]:
if SmoothenedRows > ColoredRows:
  Difference = SmoothenedRows - ColoredRows
  for x in range(Difference):
    SmoothenedRows -= 1
    Smoothened = np.delete(Smoothened, SmoothenedRows, 0)
elif SmoothenedRows < ColoredRows:
  Difference = ColoredRows - SmoothenedRows 
  for x in range(Difference):
    ColoredRows -= 1
    Colored = np.delete(Colored, ColoredRows, 0)

if SmoothenedColumns > ColoredColumns:
  Difference = SmoothenedColumns - ColoredColumns
  for x in range(Difference):
    SmoothenedColumns -= 1
    Smoothened = np.delete(Smoothened, SmoothenedColumns, 1)
elif SmoothenedColumns < ColoredColumns:
  Difference = ColoredColumns - SmoothenedColumns
  for x in range(Difference):
    ColoredColumns -= 1
    Colored = np.delete(Colored, ColoredColumns, 1)

bit-AND with color image:

In [11]:
FinalImage = Smoothened & Colored